# Import necessary libararies

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from plot_metric.functions import BinaryClassification
import sys, os
import shutil
import glob
import pickle
from os import listdir
import warnings
warnings.filterwarnings("ignore")

# Os path
script_dir  = os.path.normpath(os.path.abspath("."))
source_dir  = os.path.normpath(os.path.abspath(script_dir + "/.."))
if source_dir in sys.path: sys.path.remove(source_dir)
sys.path.insert(1, source_dir)

%matplotlib inline

# Data path and load data
train_data_dir = r'/media/nghia/DATA/DATA/RRT_CNU_ENG/Processed/Train_all_cleaned/'
# test_data_dir = r'/media/nghia/DATA/DATA/RRT_CNU_ENG/Processed/Test_all_cleaned/'

train_data = pd.read_csv(train_data_dir + 'train_all_cleaned_labelled.csv')
# test_data = pd.read_csv(test_data_dir + 'test_all_cleaned_labelled.csv')

# Window Interval Processing - make input data for DL models

In [2]:
train_data

,alternative number,SBP,DBP,HR,RR,BT,diff_ts,Diagnosis,age,gender,...,ALT,Creatinin,Total bilirubin,Glucose,HCO3,Lactate,label,is_detection,is_event,ev_w_dec
0,A03NHQHX,120.0,70.0,80.0,20.0,36.2,0.0,526,62.0,1,...,10.0,2.21,0.44,99.0,25.3,1.85,0.0,0,0,0
1,A03NHQHX,120.0,80.0,64.0,20.0,36.2,18600.0,526,62.0,1,...,108.0,0.55,0.66,82.0,36.5,1.54,0.0,0,0,0
2,A03NHQHX,120.0,80.0,64.0,20.0,36.4,28800.0,526,62.0,1,...,17.0,0.86,0.62,149.0,23.7,0.97,0.0,0,0,0
3,A03NHQHX,130.0,90.0,72.0,20.0,36.1,7200.0,526,62.0,1,...,26.0,1.25,3.59,136.0,26.3,0.50,0.0,0,0,0
4,A03NHQHX,120.0,80.0,72.0,20.0,36.0,3000.0,526,62.0,1,...,20.0,1.82,18.19,240.0,30.3,1.48,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772305,GMDTJG0Q,134.0,76.0,62.0,20.0,36.5,7200.0,2242,68.0,1,...,16.0,0.51,0.98,121.0,28.1,0.94,0.0,0,0,0
772306,GMDTJG0Q,124.0,68.0,69.0,20.0,36.4,28800.0,2242,68.0,1,...,10.0,1.15,0.36,196.0,21.1,1.09,0.0,0,0,0
772307,GMDTJG0Q,125.0,69.0,66.0,20.0,36.6,21600.0,2242,68.0,1,...,20.0,0.56,0.88,141.0,27.0,0.69,0.0,0,0,0
772308,GMDTJG0Q,127.0,78.0,72.0,20.0,36.3,28800.0,2242,68.0,1,...,71.0,1.07,0.58,126.0,34.0,1.85,0.0,0,0,0


In [3]:
# patient id list
id_list = [x for x in train_data.columns if x in ['alternative number']]

################# Features selection - you can customize input features list #################
# Laboratory test
lab_list = ['pO2', 'Potassium(K+)', 'Alkaline phosphatase' ,'pCO2', 'CRP' ,'CRP2', 'pH',
 'Albumin', 'Total protein', 'PT', 'Sodium', 'WBC', 'AST', 'Hgb', 'BUN', 'aPTT',
 'Chloride', 'Plt', 'Total calcium', 'SaO2', 'ALT', 'Creatinin',
 'Total bilirubin', 'Glucose', 'HCO3', 'Lactate']

# Vital sign
sign_list = ['HR', 'SBP', 'DBP', 'RR', 'BT']

# Demographic information - defined non-time based list
dem_list = ['Diagnosis', 'age', 'gender', 'hospitalization department', 'hospitalization route', 'inpatient ward']


# define time based list
time_list = lab_list + sign_list

#########################################################

# label list
label_abn_list = [x for x in train_data.columns if x in ['label']]
label_dec_list = [x for x in train_data.columns if x in ['is_detection']]
label_event_list = [x for x in train_data.columns if x in ['is_event']]
label_event_w_dec_list = [x for x in train_data.columns if x in ['ev_w_dec']]

In [5]:
# Window pre-processing - We can customize window length, stride, scaler, task

from dataloader import *

x_t, x_d, y_dec, y_dec_onehot = ts_dataloader(train_data, time_list, dem_list, window_len=8,
                                              scaler=8, stride=8, task='detection')

100%|██████████| 18470/18470 [10:38<00:00, 28.91it/s]


# Training

In [6]:
## defince some parameters
num_classes = 2
epochs = 100
batch_size = 1024

In [16]:
# call model - MG-SimaeseTS

from model import *

model = SiamseTS_model(x_t.shape[1:], x_d.shape[1:], num_classes=num_classes)

In [17]:
import os
from datetime import datetime

# Get current date and time in the format 'MMddHHmm'
current_time = datetime.now().strftime('%m%d%H%M')

# Train the model
keras.backend.clear_session()

model_path = r'/media/nghia/DATA/DATA/RRT_CNU_ENG/Intergatrate_model/Detection/MGTS_88/'
if not os.path.exists(model_path):
    os.makedirs(model_path)

result_path = f'{model_path}/result/'
if not os.path.exists(result_path):
    os.makedirs(result_path)

logs_path = f'{model_path}/logs_{current_time}/'
if not os.path.exists(logs_path):
    os.makedirs(logs_path)

best_model_path = f'{model_path}/best_model_{current_time}.hdf5'
log_model_path = f'{model_path}/logs_{current_time}.log.csv'

if os.path.exists(best_model_path):
        print(f'Remove {best_model_path}')
        os.remove(best_model_path)
        !ls "$logs_path"
    
if os.path.exists(log_model_path):
    print(f'Remove {log_model_path}')
    os.remove(log_model_path)
    !ls "$logs_path"

# define callback
cbs = []
cbs.append(tf.keras.callbacks.ModelCheckpoint(
    filepath=best_model_path,
    save_weights_only=True,
    monitor='loss',
    mode='min',
    verbose=1,
    save_best_only=True))

cbs.append(tf.keras.callbacks.TensorBoard(log_dir=f'{logs_path}'))
cbs.append(tf.keras.callbacks.EarlyStopping(monitor = 'loss', mode='min', patience=10))
cbs.append(tf.keras.callbacks.CSVLogger(filename=log_model_path, separator=",", append=False))

model.fit(x = [x_t, x_d], y=y_dec_onehot, epochs=epochs, batch_size=batch_size, shuffle=True, callbacks=cbs)


Epoch 1/100


2023-12-30 15:20:58.321668: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-12-30 15:20:58.322924: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:234] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.6
2023-12-30 15:20:58.322937: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:237] Used ptxas at ptxas
2023-12-30 15:20:58.323003: W tensorflow/compiler/xla/stream_executor/gpu/redzone_allocator.cc:318] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-12-30 15:20:58.553493: W tenso

509/509 [==============================] - ETA: 0s - loss: 0.1677 - acc: 0.9571 - dice_coef_multi: 0.7596 - mean_acc: 0.9578 - auc: 0.9822
Epoch 1: loss improved from inf to 0.16767, saving model to /media/nghia/DATA/DATA/RRT_CNU_ENG/Intergatrate_model/Detection/MGTS_88/best_model_12301520.hdf5
509/509 [==============================] - 23s 28ms/step - loss: 0.1677 - acc: 0.9571 - dice_coef_multi: 0.7596 - mean_acc: 0.9578 - auc: 0.9822
Epoch 2/100
508/509 [============================>.] - ETA: 0s - loss: 0.0807 - acc: 0.9785 - dice_coef_multi: 0.8530 - mean_acc: 0.9791 - auc: 0.9929
Epoch 2: loss improved from 0.16767 to 0.08070, saving model to /media/nghia/DATA/DATA/RRT_CNU_ENG/Intergatrate_model/Detection/MGTS_88/best_model_12301520.hdf5
509/509 [==============================] - 14s 27ms/step - loss: 0.0807 - acc: 0.9785 - dice_coef_multi: 0.8530 - mean_acc: 0.9791 - auc: 0.9929
Epoch 3/100
509/509 [==============================] - ETA: 0s - loss: 0.0698 - acc: 0.9806 - dice_coe